In [1]:
#HIDDEN
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import ipywidgets as widgets


# Use the "Computer Modern" font for the matplotlib figures
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# Allow LaTeX in the matplotlib figures
rc('text', usetex=True)

In [2]:
#HIDDEN
def add_corner_arc(ax, line, radius=.7, color=None, text=None, text_radius=.5, text_rotatation=0, **kwargs):
    '''
    Draws an arc for p0p1p2 angle.

    Parameters
    ----------
    ax: Axis to add arc to
    line: Matplotlib line consisting of 3 points of the corner
    radius: radius to add arc
    color: color of the arc
    text: text to show on corner
    text_radius: radius to add text
    text_rotatation: extra rotation for text
    kwargs: other arguments to pass to Arc

    Returns
    -------
    arc :  Matplotlib Arc object

    Notes
    -----
    From https://stackoverflow.com/a/26417252
    Inputs:

    '''

    lxy = line.get_xydata()

    if len(lxy) < 3:
        raise ValueError('at least 3 points in line must be available')

    p0 = lxy[0]
    p1 = lxy[1]
    p2 = lxy[2]

    width = np.ptp([p0[0], p1[0], p2[0]])
    height = np.ptp([p0[1], p1[1], p2[1]])

    n = np.array([width, height]) * 1.0
    p0_ = (p0 - p1) / n
    p1_ = (p1 - p1)
    p2_ = (p2 - p1) / n 

    theta0 = -get_angle(p0_, p1_)
    theta1 = -get_angle(p2_, p1_)

    if color is None:
        # Uses the color line if color parameter is not passed.
        color = line.get_color() 
    arc = ax.add_patch(mpl.patches.Arc(p1, width * radius, height * radius, 0, theta0, theta1, color=color, **kwargs))

    if text:
        v = p2_ / np.linalg.norm(p2_)
        if theta0 < 0:
            theta0 = theta0 + 360
        if theta1 < 0:
            theta1 = theta1 + 360
        theta = (theta0 - theta1) / 2 + text_rotatation
        pt = np.dot(rotation_transform(theta), v[:,None]).T * n * text_radius
        pt = pt + p1
        pt = pt.squeeze()
        ax.text(pt[0], pt[1], text,
                horizontalalignment='left',
                verticalalignment='top',)

    return arc


def get_angle(p0, p1=np.array([0,0]), p2=None):
    '''
    Compute angle (in degrees) for p0p1p2 corner.

    Parameters
    ----------
    p0,p1,p2: Points in the form of [x,y]

    Returns
    -------
    angle :  angle in degrees

    Notes
    -----
    From https://stackoverflow.com/a/26417252
    Inputs:
    '''
    if p2 is None:
        p2 = p1 + np.array([1, 0])
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return np.degrees(angle)

def rotation_transform(theta):
    '''
    Rotation matrix given theta.

    Parameters
    ----------
    theta: theta (in degrees)

    Returns
    -------
    A :  Rotation matrix for a given theta

    Notes
    -----
    From https://stackoverflow.com/a/26417252
    '''
    theta = np.radians(theta)
    A = [[np.math.cos(theta), -np.math.sin(theta)],
         [np.math.sin(theta), np.math.cos(theta)]]
    return np.array(A)

In [3]:
#HIDDEN
def f(theta, P):
    '''
    Function to draw the interactive widget.

    Parameters
    ----------
    theta : angle in degrees
    P: Load in N

    Returns
    -------
    None.

    Notes
    -----
    None.
    '''

    # Create figure and subplots in a grid that fits three subplots
    fig = plt.figure(figsize=(9, 5), dpi=150)
    gs = fig.add_gridspec(2, 2)
    ax1 = fig.add_subplot(gs[:, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 1])

    # Geometry definition
    L = 2  # Plot width
    l = 1  # Sample width
    H = 2.4  # Plot height
    h = 1.6  # Sample height
    x0 = - l / 2  # Bottom left corner x-coordinate
    y0 = - h / 2  # Bottom left corner y-coordinate

    # Calculate normal and shear stresses
    sig = P / l  # Applied stress
    sig_theta = round(sig * np.cos(np.deg2rad(theta))**2, 2)  # Normal stress component at the theta plane
    sig_xx = round(sig * np.sin(np.deg2rad(theta))**2, 2)  # Normal stress in the x-direction after rotation.
    sig_yy = round(sig * np.cos(np.deg2rad(theta))**2, 2)  # Normal stress in the y-direction after rotation.
    sig_max = sig  # Maximum normal stress
    tau_xy = tau_yx = tau_theta = round(sig * np.sin(np.deg2rad(theta)) * np.cos(np.deg2rad(theta)), 2)  # Shear stress component at the theta plane
    tau_xy_max = round(sig * np.sin(np.deg2rad(45)) * np.cos(np.deg2rad(45)), 2)  # Maximum shear stress

    # ax1: Original and rotated elements, oblique plane and forces applied and the resulting stresses
    # Add subplot identification
    ax1.text(-0.925, 1.125, r'a', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))
    # Draw the original element
    ax1.add_patch(mpl.patches.Rectangle((-0.15, -0.15), 0.3, 0.3, zorder=350, fc='none', ec='k', lw=1.25, ls=':'))
    # Draw the rotated element
    ax1.add_patch(mpl.patches.Rectangle((-0.15, -0.15), 0.3, 0.3, zorder=300, fc='none', ec='orangered', lw=1.25, ls='-',
                                              transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax1.transData))
    # Draw the specimen body
    ax1.add_patch(mpl.patches.Rectangle((x0, y0), l, h, zorder=-1, fc='0.6', ec='k'))
    # Add covers to hide rotated lines
    ax1.add_patch(mpl.patches.Rectangle((x0-0.1, h/2 + 0.01), l + 0.2, 0.38, zorder=5, color='w'))
    ax1.add_patch(mpl.patches.Rectangle((x0-0.1, -h/2 - 0.01), l + 0.1, -0.38, zorder=5, color='w'))

    # Create x-coordinates for rotated line
    xs = np.linspace(- l / 2, l / 2, 100)
    ax1.plot(xs, xs * np.tan(np.deg2rad(theta)), c='orangered', lw=2)

    # Draw arrows of applied load
    ax1.arrow(0, -h/2, 0, -0.3, head_width=0.05, head_length=0.09, zorder=200, fc='k', width=0.01)
    ax1.arrow(0, h/2, 0, 0.3, head_width=0.05, head_length=0.09, zorder=200, fc='k', width=0.01)
    ax1.text(0.1, -h/2 - 0.2, f'$P = {P}$' + r' $\textrm{N}$', zorder=200)
    ax1.text(0.1, +h/2 + 0.2, f'$P = {P}$' + r' $\textrm{N}$', zorder=200)

    # Reference line fo non-rotated element
    ax1.axhline(0, l/L/2, 1 - l/L/2, ls=':', c='k')

    ax1.add_patch(mpl.patches.Arc((0, 0), 0.65, 0.65, theta2=theta))
    ax1.text(0.44 * np.cos(np.deg2rad(theta / 2)), 0.44 * np.sin(np.deg2rad(theta / 2)), r'$\theta = $' + f'{theta:.3}°',
             bbox=dict(boxstyle="round", fc="w", alpha=0.8))

    # Draw arrows of applied stress in the non-rotated element
    ax1.arrow(0, 0.15, 0, 0.1, head_width=0.03, head_length=0.045, zorder=200, fc='k', width=0.005)
    ax1.arrow(0, -0.15, 0, -0.1, head_width=0.03, head_length=0.045, zorder=200, fc='k', width=0.005)
    ax1.text(0.04, 0.25, '$\sigma_y$')
    ax1.text(0.04, -0.3, '$\sigma_y$')

    # Add coordinate system
    ax1.annotate('', xy=(-0.6, -1.1), xytext=(-0.9225, -1.1),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=0.5, headwidth=4, headlength=3), zorder=100)
    ax1.annotate('', xy=(-0.9, -0.8), xytext=(-0.9, -1.12),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=0.5, headwidth=4, headlength=3), zorder=100)
    ax1.plot(-0.9, -1.1, 'o', c='k', markersize=4)
    ax1.text(-0.575, -1.12, '$x$', fontsize=12, ha='center', va='center', zorder=100)
    ax1.text(-0.95, -0.8, '$y$', fontsize=12, ha='center', va='center', zorder=100)

    # Set axis aspect and limits
    ax1.set_aspect('equal')
    ax1.set_xlim(-L/2, L/2)
    ax1.set_ylim(-H/2, H/2)

    # Remove the tichs from the axes
    ax1.set(xticks=[], xticklabels=[], yticks=[], yticklabels=[])

    # ax2: Rotated element and stress state on the rotated element
    # Add subplot identification
    ax2.text(-0.345, 0.345, r'b', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))

    # Draw the rotated texts
    if sig_yy / sig_max > 0:
        ax2.text(0.06, 0.3, r'$\sigma_y^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(0.06, -0.3, r'$\sigma_y^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
    if sig_xx / sig_max > 0:
        ax2.text(0.3, -0.07, r'$\sigma_x^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(-0.3, -0.07, r'$\sigma_x^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
    if tau_xy / tau_xy_max > 0:
        ax2.text(0.2, 0.12, r'$\tau_{xy}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(-0.11, -0.21, r'$\tau_{yx}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(-0.2, -0.11, r'$\tau_{xy}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')
        ax2.text(0.11, 0.21, r'$\tau_{yx}^\theta$', va='center', ha='center', transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData, color='orangered')

    # Draw the rotated element
    ax2.add_patch(mpl.patches.Rectangle((-0.15, -0.15), 0.3, 0.3, zorder=300, fc='none', ec='orangered', lw=1.75, ls='-',
                                              transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData))
    # Draw the rotated arrows
    if tau_xy > 0:
        ax2.arrow(0.18, -0.15 * tau_xy / tau_xy_max / 2, 0, (0.3 - 0.045) * tau_xy / tau_xy_max / 2, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
        ax2.arrow(-0.18, 0.15 * tau_xy / tau_xy_max / 2, 0, (-0.3 + 0.045) * tau_xy / tau_xy_max / 2, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
        ax2.arrow(-0.15* tau_xy / tau_xy_max / 2, 0.18, (0.3 - 0.045)* tau_xy / tau_xy_max / 2, 0, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
        ax2.arrow(0.15* tau_xy / tau_xy_max / 2, -0.18, (-0.3 + 0.045)* tau_xy / tau_xy_max / 2, 0, head_width=0.03, head_length=0.045 * tau_xy / tau_xy_max / 2, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    if sig_yy > 0:
        ax2.arrow(0, 0.22, 0, 0.1 * sig_yy / sig_max, head_width=0.03, head_length=0.045 * sig_yy / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
        ax2.arrow(0, -0.22, 0, -0.1 * sig_yy / sig_max, head_width=0.03, head_length=0.045 * sig_yy / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
    if sig_xx > 0:
        ax2.arrow(0.22, 0, 0.1 * sig_xx / sig_max, 0, head_width=0.03, head_length=0.045 * sig_xx / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)
        ax2.arrow(-0.22, 0, -0.1 * sig_xx / sig_max, 0, head_width=0.03, head_length=0.045 * sig_xx / sig_max, zorder=200, fc='orangered', edgecolor='orangered', width=0.005,
                 transform=mpl.transforms.Affine2D().rotate_deg_around(*(0,0), theta) + ax2.transData)

    # Set axis aspect and limits
    ax2.set_aspect('equal')
    ax2.set_xlim(-0.4, 0.4)
    ax2.set_ylim(-0.4, 0.4)

    # Remove the tichs from the axes
    ax2.set(xticks=[], xticklabels=[], yticks=[], yticklabels=[])

    # ax3: Summary of the stresses and stress states
    # Add subplot identification
    ax3.text(-0.182, 0.935, r'c', va='center', ha='center',
             bbox=dict(boxstyle="circle", fc="w", alpha=0.8))

    # Draw the stress states
    ax3.text(0.255, 0.95, r'$P = ' + f'{P}$' + r' $\textrm{N}$', ha='center', va='top')
    ax3.text(0.255, 0.85, r'$\sigma_p^\theta = ' + f' {sig_theta:4.4} ' + r'\ \textrm{MPa}$', ha='center', va='top', color='orangered')
    ax3.text(0.255, 0.725, r'$\tau_p^\theta = ' + f' {tau_theta:4.4} ' + r'\ \textrm{MPa}$', ha='center', va='top', color='orangered')
    ax3.text(0.255, 0.575, fr'$\mathbf{{\sigma}}_{{ij}}^\theta = \left[\matrix{{\sigma_{{x}}^\theta & \tau_{{xy}}^\theta \cr \tau_{{yx}}^\theta & \sigma_{{y}}^\theta }} \right] = \left[ \matrix{{{sig_xx:4.4} & {tau_xy:4.4} \cr {tau_yx:4.4} & {sig_yy:4.4} }} \right]$', ha='center', va='top', color='orangered')
    ax3.text(0.255, 0.3, fr'$\mathbf{{\sigma}}_{{ij}} = \left[\matrix{{\sigma_{{x}} & \tau_{{xy}} \cr \tau_{{yx}} & \sigma_{{y}} }} \right] = \left[ \matrix{{0.0 & 0.0 \cr 0.0 & 500.0 }} \right]$', ha='center', va='top')

    # Remove the tichs from the axes
    ax3.set(xticks=[], xticklabels=[], yticks=[], yticklabels=[])

    # Set axis aspect and limits
    ax3.set_aspect('equal')
    ax3.set_xlim(-0.25, .75)

    # Organize the layout and adjust distance between subplots
    plt.tight_layout()
    plt.subplots_adjust(wspace=-0.4)

    # Show the plot
    plt.show()

# Create slider for the rotation angle
theta_slider = widgets.FloatSlider(description=r'$\theta$ [°]',
                                   value=0,
                                   min=0,
                                   max=90,
                                   step=0.1,
                                   readout_format='.2f',
                                   continuous_update=False)

# Create slider for the load applied
P_slider = widgets.IntSlider(description=r'P [N]',
                             value=500,
                             min=1,
                             max=500,
                             step=1,
                             continuous_update=False)

# Create the interactive plot
interactive_plot = widgets.interactive(f, theta=theta_slider, P=P_slider)

# Ensure that the image is drawn before user interact with the widget
interactive_plot.update()

# Re-organize the output, putting the sliced on the bottom of the figure
output = widgets.VBox([interactive_plot.children[-1],
                      theta_slider, P_slider])
output.layout = widgets.Layout(display='flex',
                               flex_flow='column',
                               align_items='center',
                               align_content='center',
                               justify_content='center',
                               width='100%',
                               height='100%')

# Draw the output
output